### Step 1: Data Acquisition, Integration and Preparation

#### 1.1 Explore and Prepare the most-popular_global_alltime.xlsx downloaded from https://www.netflix.com/tudum/top10/most-popular.
- Split the runtime_override_flag column into is_stagerred_launch column and is_live_title column
- Split the episode_launch_dtls into max_episodes_launched column and num_countries_total column

In [2]:
import pandas as pd

# 1) Load your base table
df = pd.read_excel('Datasets/most-popular_global_alltime.xlsx')

df.head()

c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,runtime,weekly_views,cumulative_weeks_in_top_10,is_staggered_launch,runtime_override_flag,episode_launch_dtls
0,2025-05-25,Films (English),1,Fear Street: Prom Queen,NaN,16000000,1.5000,10700000.0,1,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN
1,2025-05-25,Films (English),2,Puss in Boots: The Last Wish,NaN,14800000,1.7000,8700000.0,4,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN
2,2025-05-25,Films (English),3,Nonnas,NaN,16000000,1.9000,8400000.0,3,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN
3,2025-05-25,Films (English),4,Untold: The Fall of Favre,NaN,7300000,1.0667,6800000.0,1,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN
4,2025-05-25,Films (English),5,Instant Family,NaN,10600000,1.9833,5300000.0,2,False,"[{is_staggered_launch=0, is_live_title=0}]",NaN


In [ ]:
import re
import json

# 1) Split the runtime_override_flag column via regex into two columns:
#    is_staggered_launch and is_live_title
def parse_runtime_flags(s):
    """
    From a string like "[{is_staggered_launch=1, is_live_title=0}]" 
    extract the two digits.
    """
    if pd.isna(s):
        return pd.Series({'is_staggered_launch': None, 'is_live_title': None})
    # regex look for "is_staggered_launch=<digit>" and "is_live_title=<digit>"
    stagger = re.search(r'is_staggered_launch=(\d)', s)
    live    = re.search(r'is_live_title=(\d)', s)
    return pd.Series({
        'is_staggered_launch': int(stagger.group(1)) if stagger else None,
        'is_live_title':       int(live.group(1))    if live    else None
    })

# Apply the parsing function to the runtime_override_flag column
runtime_flags = df['runtime_override_flag'].apply(parse_runtime_flags)
df = pd.concat([df, runtime_flags], axis=1)

# 2) Split the episode_launch_dtls column via json.loads into two columns:
#   max_episodesLaunched and num_countries_total
def parse_episode_details(s):
    """
    From a JSON-array-string like:
      '[{"episodesLaunched":12,"countryList":["AZ","NG",...]}, ...]'
    compute:
      - max_episodesLaunched: the largest episodesLaunched value
      - num_countries_total: total count of all countryList entries across batches
    """
    if pd.isna(s):
        return pd.Series({'max_episodesLaunched': None, 'num_countries_total': None})
    try:
        arr = json.loads(s)   # valid JSON as‐is
        # extract the episodesLaunched values
        ep_counts = [entry['episodesLaunched'] for entry in arr if 'episodesLaunched' in entry]
        # count total number of country codes across all lists
        total_countries = sum(len(entry.get('countryList', [])) for entry in arr)
        return pd.Series({
            'max_episodesLaunched': max(ep_counts) if ep_counts else None,
            'num_countries_total':  total_countries
        })
    except json.JSONDecodeError:
        # fallback if parsing fails
        return pd.Series({'max_episodesLaunched': None, 'num_countries_total': None})

# Apply the parsing function to the episode_launch_dtls column
episode_details = df['episode_launch_dtls'].apply(parse_episode_details)
df = pd.concat([df, episode_details], axis=1)

# 3) Drop the raw nested columns
df = df.drop(columns=['runtime_override_flag', 'episode_launch_dtls'])

# 4) Quick check
print(df[['is_staggered_launch','is_live_title',
          'max_episodesLaunched','num_countries_total']].head())

# 5) Save for Power BI
df.to_csv('Datasets/most-popular_global_alltime.csv', index=False)


   is_staggered_launch  is_staggered_launch  is_live_title  \
0                False                  0.0            0.0   
1                False                  0.0            0.0   
2                False                  0.0            0.0   
3                False                  0.0            0.0   
4                False                  0.0            0.0   

   max_episodesLaunched  num_countries_total  
0                   NaN                  NaN  
1                   NaN                  NaN  
2                   NaN                  NaN  
3                   NaN                  NaN  
4                   NaN                  NaN  


#### 1.2 Collecting data for omdb_results.csv from The Open Movie Database (OMDb) API over the course of three days.
- Since the API has a 1,000 records limit and as seen below we have 2,634 unique titles in the most-popular_global_alltime.csv we need to fetch metadat like genre for each of the unique title.

In [4]:
df = pd.read_csv('Datasets/most-popular_global_alltime.csv').rename(columns={'show_title':'title'})
unique_titles = df['title'].dropna().unique()
print(f"Unique titles: {len(unique_titles)}")

Unique titles: 2632


In [ ]:
#pip install requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import time
import os

# --- Load Netflix titles ---
df = pd.read_csv('Datasets/most-popular_global_alltime.csv').rename(columns={'show_title':'title'})
unique_titles = df['title'].dropna().unique()

# --- OMDb API Config ---
API_KEY = 'c83a0c4'  # Your activated OMDb API key
API_URL = "http://www.omdbapi.com/"
MAX_REQUESTS = 1000
SLEEP_TIME = 0.2  # Sleep time between requests
SAVE_INTERVAL = 50  # Save every 50 requests
OUTPUT_FILE = "omdb_results.csv"

# --- Load already processed data ---
if os.path.exists(OUTPUT_FILE):
    existing_df = pd.read_csv(OUTPUT_FILE)
    processed_titles = set(existing_df['title'].dropna().unique())
    movie_data = existing_df.to_dict('records')
    print(f"Resuming from previous file: {len(processed_titles)} titles already processed.")
else:
    processed_titles = set()
    movie_data = []

# --- Start requesting OMDb data ---
requests_made = 0

for raw_title in unique_titles:
    title = raw_title.strip()

    if title in processed_titles:
        continue

    params = {
        't': title,
        'apikey': API_KEY
    }

    try:
        response = requests.get(API_URL, params=params, timeout=5)
        data = response.json()

        if data.get('Response') == 'True':
            movie_data.append({
                'title': title,
                'imdb_title': data.get('Title'),
                'genre': data.get('Genre'),
                'released': data.get('Released'),
                'country': data.get('Country'),
                'imdb_rating': data.get('imdbRating'),
                'year': data.get('Year'),
                'type': data.get('Type')
            })
        else:
            # Store title even if data is missing (e.g., not found)
            movie_data.append({
                'title': title,
                'imdb_title': None,
                'genre': None,
                'released': None,
                'country': None,
                'imdb_rating': None,
                'year': None,
                'type': None
            })

        requests_made += 1

        # Save progress every few requests
        if requests_made % SAVE_INTERVAL == 0:
            print(f"Saving at {requests_made} requests...")
            pd.DataFrame(movie_data).to_csv(OUTPUT_FILE, index=False)

        # Stop if request limit reached
        if requests_made >= MAX_REQUESTS:
            print(f"\n--- Daily request limit ({MAX_REQUESTS}) reached ---")
            break

        time.sleep(SLEEP_TIME)  # Avoid being rate-limited

    except Exception as e:
        print(f"Error for '{title}': {e}")
        continue

# --- Final save ---
pd.DataFrame(movie_data).to_csv(OUTPUT_FILE, index=False)
print(f"\n✅ Finished! Saved {len(movie_data)} records to '{OUTPUT_FILE}'.")


Resuming from previous file: 2000 titles already processed.
Saving at 50 requests...
Saving at 100 requests...
Saving at 150 requests...
Saving at 200 requests...
Saving at 250 requests...
Saving at 300 requests...
Saving at 350 requests...
Saving at 400 requests...
Saving at 450 requests...
Saving at 500 requests...
Saving at 550 requests...
Saving at 600 requests...

✅ Finished! Saved 2634 records to 'omdb_results.csv'.


#### 1.3 Merging the OMDb API dataset that have the title's and their metadata with the most-popular_global_alltime.csv dataset 
- Merged using merge key as the title and validating the title by categories.
- If the title and category of the most-popular_global_alltime.csv does not match to the omdb_results.csv dataset's title and type then we do not merge with that record.
- We save the merged dataset as merged_global_omdb_strict_flagged.csv

In [5]:
# === Load datasets ===
df_global = pd.read_csv("Datasets/most-popular_global_alltime.csv")
df_omdb = pd.read_csv("Datasets/omdb_results.csv")

# === Clean titles ===
df_global['title_clean'] = df_global['show_title'].str.strip().str.lower()
df_omdb['title_clean'] = df_omdb['title'].str.strip().str.lower()

# === Map Netflix categories to types ===
category_map = {
    'Films (English)': 'movie',
    'Films (Non-English)': 'movie',
    'TV (English)': 'series',
    'TV (Non-English)': 'series'
}
df_global['netflix_type'] = df_global['category'].map(category_map)

# === Filter OMDb to valid types only ===
df_omdb = df_omdb[df_omdb['type'].isin(['movie', 'series'])].copy()

# === Create composite keys ===
df_global['merge_key'] = df_global['title_clean'] + "-" + df_global['netflix_type']
df_omdb['merge_key'] = df_omdb['title_clean'] + "-" + df_omdb['type']

# === Drop duplicate OMDb entries on merge_key ===
df_omdb = df_omdb.drop_duplicates(subset='merge_key')

# === Rename title column in OMDb ===
df_omdb = df_omdb.rename(columns={'title': 'title_omdb'})

# === Perform left join ===
df_merged = pd.merge(
    df_global,
    df_omdb,
    on='merge_key',
    how='left',
    suffixes=('', '_omdb')
)

# === Add metadata match flag ===
df_merged['metadata_matched'] = df_merged['genre'].notna()

# === Save final result ===
df_merged.to_csv("Datasets/merged_global_omdb_strict_flagged.csv", index=False)
print(f" Final merged dataset saved: merged_global_omdb_strict_flagged.csv")
print(f" Total rows: {len(df_merged)} | Metadata matched: {df_merged['metadata_matched'].sum()} | Unmatched: {(~df_merged['metadata_matched']).sum()}")

# Count how many titles had no metadata match
missing_count = (~df_merged['metadata_matched']).sum()

print(f" Titles with no OMDb metadata: {missing_count}")

# List all column names
print(" All columns in df_merged:")
for col in df_merged.columns:
    print(f" - {col}")

 Final merged dataset saved: merged_global_omdb_strict_flagged.csv
 Total rows: 8160 | Metadata matched: 7233 | Unmatched: 927
 Titles with no OMDb metadata: 927
 All columns in df_merged:
 - week
 - category
 - weekly_rank
 - show_title
 - season_title
 - weekly_hours_viewed
 - runtime
 - weekly_views
 - cumulative_weeks_in_top_10
 - is_staggered_launch
 - is_staggered_launch.1
 - is_live_title
 - max_episodesLaunched
 - num_countries_total
 - title_clean
 - netflix_type
 - merge_key
 - title_omdb
 - imdb_title
 - genre
 - released
 - country
 - imdb_rating
 - year
 - type
 - title_clean_omdb
 - metadata_matched


#### 1.4 Cleaning up final merged dataset
- The merged dataset has duplicate columns and columns that are not needed after the merge, so we drop them.
- Reorder the columns 
- We save the final dataset with the correct columns.

In [6]:
df_merged = pd.read_csv("Datasets/merged_global_omdb_strict_flagged.csv")

# Rename columns as needed
df_merged_cleaned = df_merged.rename(columns={
    'country': 'origin_country',
    'year': 'release_year'
})

# Select and reorder final columns
final_columns = [
    'week',
    'category',
    'weekly_rank',
    'show_title',
    'season_title',
    'weekly_hours_viewed',
    'runtime',
    'weekly_views',
    'cumulative_weeks_in_top_10',
    'is_staggered_launch',
    'is_live_title',
    'max_episodesLaunched',
    'num_countries_total',
    'genre',
    'origin_country',
    'release_year',
    'imdb_rating',
    'metadata_matched'
]

# Drop all other columns and reindex
df_final = df_merged_cleaned[final_columns]

# Save to CSV
df_final.to_csv("Datasets/merged_global_omdb_final.csv", index=False)
print("✅ Final cleaned dataset saved as: merged_global_omdb_final.csv")
print(f"📊 Final shape: {df_final.shape}")


✅ Final cleaned dataset saved as: merged_global_omdb_final.csv
📊 Final shape: (8160, 18)


#### 1.5 As we saw above the final merged_global_omdb_final.csv dataset has 927 records with missing metedata which the OMDb API could not provide.
- I filtered the merged_global_omdb_final.csv uing the metadata_matched columns with false value which were 927 rows without the metadata filled.
- Therefore, I manually entered the data from the netflix and IMDb websites and confirming it is the exact title mentioned from the netflix site.

#### The final dataset after the data acquisition and data integration is present in the folder called: final_netflix_omdb_dataset.csv

Note to self: 
- Accidently deleted the dataset somehow by rerunning on it and it was with the hand-filled 927 records and uncleaned dataset which was the original dataset loaded above called the 'final_netflix_omdb_dataset.csv but now we only have the one with the cleaned special character and unneccessary columns so its not really a problem but just keep that in mind and do not re-run the following cells.

In [ ]:
# Load the final dataset with OMDb metadata
# We use ISO-8859-1 encoding to handle latin based special characters in titles
df = pd.read_csv('Datasets/final_netflix_omdb_dataset.csv', encoding='ISO-8859-1')

df.head(42)

,week,category,weekly_rank,show_title,season_title,weekly_hours_viewed,runtime,weekly_views,cumulative_weeks_in_top_10,is_staggered_launch,is_live_title,max_episodesLaunched,num_countries_total,genre,origin_country,release_year,imdb_rating,metadata_matched
0,25/05/2025,Films (English),1,Fear Street: Prom Queen,NaN,16000000,1.5000,10700000.0,1,False,0.0,NaN,NaN,"Horror, Mystery, Thriller",United States,2025,5.1,True
1,25/05/2025,Films (English),2,Puss in Boots: The Last Wish,NaN,14800000,1.7000,8700000.0,4,False,0.0,NaN,NaN,"Animation, Action, Adventure","United States, Japan",2022,7.8,True
2,25/05/2025,Films (English),3,Nonnas,NaN,16000000,1.9000,8400000.0,3,False,0.0,NaN,NaN,Comedy,United States,2025,6.9,True
3,25/05/2025,Films (English),4,Untold: The Fall of Favre,NaN,7300000,1.0667,6800000.0,1,False,0.0,NaN,NaN,"Crime, Sport",United States,2025,6.0,False
4,25/05/2025,Films (English),5,Instant Family,NaN,10600000,1.9833,5300000.0,2,False,0.0,NaN,NaN,"Comedy, Drama",United States,2018,7.3,True
5,25/05/2025,Films (English),6,Havoc,NaN,9000000,1.7833,5000000.0,5,False,0.0,NaN,NaN,"Action, Crime, Drama","United Kingdom, United States",2025,NaN,True
6,25/05/2025,Films (English),7,A Deadly American Marriage,NaN,7500000,1.7167,4400000.0,3,False,0.0,NaN,NaN,"Documentary, Crime",United Kingdom,2025,NaN,True
7,25/05/2025,Films (English),8,Dr. Seuss' The Cat in the Hat,NaN,5500000,1.3667,4000000.0,1,False,0.0,NaN,NaN,"Animation, Short",United States,2012,NaN,True
8,25/05/2025,Films (English),9,Air Force Elite: Thunderbirds,NaN,5700000,1.5167,3800000.0,1,False,0.0,NaN,NaN,Documentary,United States,2025,NaN,True
9,25/05/2025,Films (English),10,The Wild Robot,NaN,5600000,1.7000,3300000.0,1,False,0.0,NaN,NaN,"Animation, Sci-Fi","United States, Japan",2024,8.2,True


#### We can see that the categories in the dataset are:
- 'Films (English)': 'movie',
- 'Films (Non-English)': 'movie',
- 'TV (English)': 'series',
- 'TV (Non-English)': 'series'
#### Each week like the one above 25/05/2025 in dd-mm-yyyy format is repeated for each of the four categories which looks like the following:
- This shows that each week is repeated for each category and the titles in that category are ranked from 1 - 10. 
- Therefore, the titles are repeated.
- Moreover, cumulative_weeks_in_top_10 column tracks which titles are repeated in rank in top 10 on netflix and is a measure of popularity for a title if its cumulative_weeks_in_top_10 value is more than 1.
- Each title has metadata that we got from the omdb api and by filling out some records manually.
##### Goal: Use the `cumulative_weeks_in_top_10` column for popularity and the titles metadata like genre can be used for testing hypothesis using chi-square or anova or linear regression or logistic regression, whatever fits.

#### 1.6.1 Cleaning the dataset
- Before we use inferential statistics on the data, we need to clean the data since we are using ISO-8859-1 so lets search for special characters and clean them properly through replacement or removing.
- Explore the dataset and drop columns with predominantly missing data.
- Save the cleaned dataset.

In [11]:
import pandas as pd
import re

# Load dataset
df = pd.read_csv('Datasets/final_netflix_omdb_dataset.csv', encoding='ISO-8859-1')

# Quick overview
print("Initial shape:", df.shape)
print(df.info())

Initial shape: (8160, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8160 entries, 0 to 8159
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   week                        8160 non-null   object 
 1   category                    8160 non-null   object 
 2   weekly_rank                 8160 non-null   int64  
 3   show_title                  8160 non-null   object 
 4   season_title                3948 non-null   object 
 5   weekly_hours_viewed         8160 non-null   int64  
 6   runtime                     4080 non-null   float64
 7   weekly_views                4080 non-null   float64
 8   cumulative_weeks_in_top_10  8160 non-null   int64  
 9   is_staggered_launch         8160 non-null   bool   
 10  is_live_title               4080 non-null   float64
 11  max_episodesLaunched        133 non-null    float64
 12  num_countries_total         133 non-null    float64
 13  genre  

In [13]:
# Function to clean special characters
def clean_special_chars(text):
    if pd.isna(text):
        return text
    return re.sub(r'[^\x00-\x7F]+', '', str(text)).strip()

# Apply to relevant columns
columns_to_clean = ['show_title', 'season_title', 'genre', 'origin_country', 'release_year']
for col in columns_to_clean:
    df[col] = df[col].apply(clean_special_chars)


In [ ]:
# Drop columns with more than 70% missing values
threshold = 0.7
missing_ratio = df.isnull().mean()
cols_to_drop = missing_ratio[missing_ratio > threshold].index.tolist()
print(f"Dropping columns (>{int(threshold*100)}% missing): {cols_to_drop}")

df_cleaned = df.drop(columns=cols_to_drop)

Dropping columns (>70% missing): ['max_episodesLaunched', 'num_countries_total']


In [ ]:
# Save cleaned dataset
output_path = 'Datasets/final_netflix_omdb_dataset_cleaned.csv'
df_cleaned.to_csv(output_path, index=False)
print(f" Cleaned dataset saved to: {output_path}")
print(" Final shape:", df_cleaned.shape)

 Cleaned dataset saved to: Datasets/final_netflix_omdb_dataset_cleaned.csv
 Final shape: (8160, 16)


In [4]:
import pandas as pd
import re

# Load the cleaned dataset
df = pd.read_csv('Datasets/final_netflix_omdb_dataset_cleaned.csv')

# --- Step 1: Drop the 'metadata_matched' column ---
df = df.drop(columns=['metadata_matched'])

# --- Step 2: Fix 'release_year' formatting for stuck-together years ---
def fix_year_format(year):
    if pd.isna(year):
        return year
    year_str = str(year)
    if re.fullmatch(r'\d{8}', year_str):  # Exactly 8 digits
        return f"{year_str[:4]}-{year_str[4:]}"
    return year_str

df['release_year'] = df['release_year'].apply(fix_year_format)

# --- Step 3: Keep only the first country from 'origin_country' ---
def get_primary_country(country_str):
    if pd.isna(country_str):
        return country_str
    return country_str.split(',')[0].strip()

df['origin_country'] = df['origin_country'].apply(get_primary_country)

# --- Step 4: Save final dataset ---
output_path = 'Datasets/netflix_omdb_dataset.csv'
df.to_csv(output_path, index=False)
print(f"✅ Final dataset saved to: {output_path}")

# --- Step 5: Show sample ---
print("📊 Final cleaned dataset preview:")
print(df.head(10))


✅ Final dataset saved to: Datasets/netflix_omdb_dataset.csv
📊 Final cleaned dataset preview:
         week         category  weekly_rank                     show_title  \
0  25/05/2025  Films (English)            1        Fear Street: Prom Queen   
1  25/05/2025  Films (English)            2   Puss in Boots: The Last Wish   
2  25/05/2025  Films (English)            3                         Nonnas   
3  25/05/2025  Films (English)            4      Untold: The Fall of Favre   
4  25/05/2025  Films (English)            5                 Instant Family   
5  25/05/2025  Films (English)            6                          Havoc   
6  25/05/2025  Films (English)            7     A Deadly American Marriage   
7  25/05/2025  Films (English)            8  Dr. Seuss' The Cat in the Hat   
8  25/05/2025  Films (English)            9  Air Force Elite: Thunderbirds   
9  25/05/2025  Films (English)           10                 The Wild Robot   

  season_title  weekly_hours_viewed  runtime  we